In [400]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm as tqdm

%matplotlib inline

import torch
import torchvision

import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

from torch.utils.data import Dataset, DataLoader

In [401]:
m = 2000 
size = 500
desired_num = size + 1000 


In [402]:
np.random.seed(1)

y = np.concatenate((np.zeros(500),np.ones(500),np.ones(500)*2))
#y = np.random.randint(0,3,6000)
idx= []
for i in range(3):
    print(i,sum(y==i))
    idx.append(y==i)

In [403]:
x = np.zeros((1500,))


np.random.seed(12)
x[idx[0]] = np.random.uniform(low =-1,high =0,size= sum(idx[0]))
x[idx[1]] = np.random.uniform(low =0,high =1,size= sum(idx[1]))
x[idx[2]] = np.random.uniform(low =2,high =3,size= sum(idx[2]))

In [404]:
for i in range(3):
    y1= np.zeros(x[idx[i]].shape[0])
    plt.scatter(x[idx[i]],y1,label="class_"+str(i))
plt.legend()

In [405]:
mean_x = np.mean(x,axis=0,keepdims=True)
std_x = np.std(x,axis=0,keepdims=True)
x = ( x -  mean_x ) / std_x

In [406]:
for i in range(3):
    y1= np.zeros(x[idx[i]].shape[0])
    plt.scatter(x[idx[i]],y1,label="class_"+str(i))
plt.legend()

In [407]:
foreground_classes = {'class_0','class_1' }

background_classes = {'class_2'}

In [408]:
mosaic_list_of_images =[]
mosaic_label = []
fore_idx=[]
for j in tqdm(range(desired_num)):
    np.random.seed(j)
    fg_class  = np.random.randint(0,2)
    fg_idx = np.random.randint(0,m)
    a = []
    for i in range(m):
        if i == fg_idx:
            b = np.random.choice(np.where(idx[fg_class]==True)[0],size=1)
            a.append(x[b])
#             print("foreground "+str(fg_class)+" present at " + str(fg_idx))
        else:
            bg_class = np.random.randint(2,3)
            b = np.random.choice(np.where(idx[bg_class]==True)[0],size=1)
            a.append(x[b])
#             print("background "+str(bg_class)+" present at " + str(i))
    a = np.concatenate(a,axis=0)
    mosaic_list_of_images.append(np.reshape(a,(m,1)))
    mosaic_label.append(fg_class)
    fore_idx.append(fg_idx)

In [409]:
mosaic_list_of_images = np.concatenate(mosaic_list_of_images,axis=1).T

In [410]:
mosaic_list_of_images.shape, mosaic_list_of_images[0]

In [411]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list_of_images, mosaic_label, fore_idx):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list_of_images
    self.label = mosaic_label
    self.fore_idx = fore_idx

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx], self.fore_idx[idx]


In [412]:
batch = 250
msd1 = MosaicDataset(mosaic_list_of_images[0:size], mosaic_label[0:size] , fore_idx[0:size])
train_loader = DataLoader( msd1 ,batch_size= batch ,shuffle=True)



batch = 250
msd2 = MosaicDataset(mosaic_list_of_images[size:], mosaic_label[size:] , fore_idx[size:])
test_loader = DataLoader( msd2 ,batch_size= batch ,shuffle=True)

In [413]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(1, 50)
    self.fc2 = nn.Linear(50,2)

    torch.nn.init.xavier_normal_(self.fc1.weight)
    torch.nn.init.zeros_(self.fc1.bias)
    torch.nn.init.xavier_normal_(self.fc2.weight)
    torch.nn.init.zeros_(self.fc2.bias)

    #torch.nn.init.xavier_normal_(self.fc3.weight)
    #torch.nn.init.zeros_(self.fc3.bias)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    # print(x.shape)
    return x

In [425]:
torch.manual_seed(13)
net = Net().double()
net = net.to("cuda")

In [426]:
criterion = nn.CrossEntropyLoss()
def my_cross_entropy(output,target,loss_criterion):
    batch = output.size(0)
    #print(batch)
    patches = output.size(1)
    classes = output.size(2) 
    output = torch.reshape(output,(batch*patches,classes))
    target = target.repeat_interleave(patches)
    loss = loss_criterion(output,target)
    #print(final_loss)
    return loss

In [427]:
optimizer = optim.Adam(net.parameters(), lr=0.1)#, momentum=0.9)

In [428]:
acti = []
loss_curi = []

epochs  = 300
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data1 in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels,_ = data1
        #print(labels.shape)
        inputs = inputs.double()
        labels = labels.long()
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs[:,:,None])



        #print(outputs.shape,inputs.shape)
        loss = my_cross_entropy(outputs, labels,criterion)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        mini = 2
        if i % mini == (mini-1):    # print every  mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / mini))
            ep_lossi.append(running_loss/mini) # loss per minibatch
            running_loss = 0.0
            
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch

    if(np.mean(ep_lossi)<=0.001):
      break
print('Finished Training')

In [429]:
correct = 0
total = 0
datx = []
labelsx = []
with torch.no_grad():
    for data in train_loader:
        images, labels,_ = data
        
        batch = images.size(0)
        patches = images.size(1)
        
        images = torch.reshape(images,(batch*patches,1))
        labels = labels.repeat_interleave(patches)
        datx.append(images)
        labelsx.append(labels)
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted.shape)
        #print(labels.shape)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the train images: %d %%' % ( 100 * correct / total))

In [430]:
datx = np.concatenate(datx,axis=0)
labelsx = np.concatenate(labelsx,axis=0)
print(datx.shape)

In [431]:

X,Y = np.meshgrid(np.arange(-2, 2.5,0.01),np.arange(-1, 1, 0.1))

data = np.c_[X.ravel(), Y.ravel()]
data = torch.tensor(data[:,0]).double()
data = data.to("cuda")
  
  
with torch.no_grad():
  Z1 = net(data[:,None])
  #print(Z1.shape)
_, Z1 = torch.max(Z1.data, 1)

  #print(Z1.shape)
Z1 = Z1.cpu().numpy()
    
Z1 = Z1.reshape(X.shape)

print(np.unique(Z1))
plt.figure(figsize=(6,8))
cax = plt.contourf(X,Y,Z1)
plt.colorbar(cax)
dummy_y = np.zeros(datx.shape[0])
scatter = plt.scatter(datx ,dummy_y,c=labelsx,cmap=plt.cm.Paired,alpha=0.8)
plt.yticks(()) #Y value is meaningless in this data. 
plt.legend(handles=scatter.legend_elements()[0], labels=["fg0","fg1"])
plt.title("classification map "+str(1000))

In [432]:
np.random.seed(2)

y_test = np.concatenate((np.zeros(500),np.ones(500)))
#y = np.random.randint(0,3,6000)
idx_t= []
for i in range(2):
    print(i,sum(y_test==i))
    idx_t.append(y_test==i)
x_test = np.zeros((1000,))


np.random.seed(12)

x_test[idx_t[0]] = np.random.uniform(low =-1,high =0,size= sum(idx_t[0]))
x_test[idx_t[1]] = np.random.uniform(low =0,high =1,size= sum(idx_t[1]))

In [433]:
x_test= (x_test - mean_x)/std_x

In [434]:
for i in range(2):
    y_dummy = np.zeros(x_test[idx_t[i]].shape[0])
    plt.scatter(x_test[idx_t[i]],y_dummy,label="class_"+str(i))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [435]:
x_test = torch.tensor(x_test).to("cuda")
outputs = net(x_test[:,None])
_, predicted = torch.max(outputs.data, 1)
print("Accuracy on True Data ", np.sum(predicted.cpu().numpy()  == y_test) /10,"%")